In [ ]:
import requests
import datetime
import json
import http.client
import pandas as pd
import dateutil.parser
import time

In [ ]:
api_url = "https://fitdance.ipzmarketing.com/api/v1"
api_key = ""
headers = {
    'Content-Type': 'application/json',
    'x-auth-token': f'{api_key}'
}

initial_campaign_ids = []
page = 1

while True:
    url = f"{api_url}/sent_campaigns?page={page}"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break

    data = response.json()

    if isinstance(data, list) and data:
        for campaign in data:
            if isinstance(campaign, dict) and 'id' in campaign:
                initial_campaign_ids.append(campaign['id'])
    else:
        break

    page += 1

max_id = max(initial_campaign_ids)
all_possible_ids = list(range(1, max_id + 1))
missing_ids = list(set(all_possible_ids) - set(initial_campaign_ids))

campaign_ids = initial_campaign_ids.copy()
for missing_id in missing_ids:
    url = f"{api_url}/sent_campaigns/{missing_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if isinstance(data, dict) and 'id' in data:
            campaign_ids.append(data['id'])

campaign_ids.sort()

print("Sent Campaign IDs:", campaign_ids)

with open('sent_campaign_ids.json', 'w') as f:
    json.dump(campaign_ids, f, indent=4)

In [ ]:

api_url = "https://fitdance.ipzmarketing.com/api/v1"
api_key = "i5iz9Vx1rYiLhbanimAKwKtmrNcAHvFCyLSFqRe_"
headers = {'x-auth-token': api_key}

# Function to fetch campaign data with retry logic
def fetch_campaign_data(campaign_id, max_retries=3, retry_delay=5):
    for retry_count in range(max_retries):
        conn = http.client.HTTPSConnection("fitdance.ipzmarketing.com")
        conn.request("GET", f"/api/v1/sent_campaigns/{campaign_id}", headers=headers)
        res = conn.getresponse()
        data = res.read()

        if res.status == 200:
            try:
                campaign_data = json.loads(data.decode("utf-8"))
                if "html" in campaign_data:
                    del campaign_data["html"]
                return campaign_data
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for campaign_id {campaign_id}: {e}")
                print(f"Response data: {data}")
                return None
        elif res.status == 429:
            print(f"Rate limit exceeded for campaign_id {campaign_id}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print(f"Error fetching data for campaign_id {campaign_id}: Status code {res.status}")
            if retry_count < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                return None

    return None

# Load campaign IDs
with open('sent_campaign_ids.json') as f:
    sent_campaign_ids = json.load(f)

# Fetch campaign data for all IDs (initial attempt)
all_campaign_data = []
missing_campaign_ids = []

for campaign_id in sent_campaign_ids:
    campaign_data = fetch_campaign_data(campaign_id)
    if campaign_data:
        all_campaign_data.append(campaign_data)
    else:
        missing_campaign_ids.append(campaign_id)

# Retry for missing campaign IDs with more retries and longer delay
if missing_campaign_ids:
    print("Retrying for missing campaign IDs...")
    for campaign_id in missing_campaign_ids:
        campaign_data = fetch_campaign_data(campaign_id, max_retries=5, retry_delay=10)
        if campaign_data:
            all_campaign_data.append(campaign_data)
            print(f"Successfully retrieved data for campaign_id {campaign_id} after retry.")
        else:
            print(f"Failed to retrieve data for campaign_id {campaign_id} even after retry.")

# Sort campaign data by ID
all_campaign_data.sort(key=lambda x: x['id'])

# Create DataFrame
df = pd.DataFrame(all_campaign_data)
print(df)

# Save campaign data
with open('campaign_data.json', 'w') as f:
    json.dump(all_campaign_data, f, indent=4)